# Breast Cancer Classification and Evaluation

Breast Cancer is a very easy binary classification dataset. It doesn't have any missing values, so it makes it a very nice and tidy dataset to showcase how to use Cyclops features.

In [1]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

from cyclops.data.slicer import SliceSpec
from cyclops.evaluate.metrics import create_metric
from cyclops.evaluate.metrics import BinaryAccuracy
from cyclops.evaluate.metrics.experimental import BinaryAUROC, BinaryAveragePrecision
from cyclops.evaluate.metrics.experimental.metric_dict import MetricDict
from cyclops.evaluate.metrics.factory import create_metric
# from cyclops import evaluator
# from datasets.arrow_dataset import Dataset


/Users/royajavadi/venvs/cyclops/lib/python3.10/site-packages/monai/transforms/io/array.py:205: UserWarning: required package for reader ITKReader is not installed, or the version doesn't match requirement.
  warnings.warn(


In [2]:
# Loading the data
breast_cancer_data = datasets.load_breast_cancer(as_frame=True)
X, y = breast_cancer_data.data, breast_cancer_data.target

### Features
Just taking a quick look at features and their stats...

In [3]:
df = breast_cancer_data.frame
df.describe().T

,count,mean,std,min,25%,50%,75%,max
mean radius,569.0,14.127292,3.524049,6.981000,11.700000,13.370000,15.780000,28.11000
mean texture,569.0,19.289649,4.301036,9.710000,16.170000,18.840000,21.800000,39.28000
mean perimeter,569.0,91.969033,24.298981,43.790000,75.170000,86.240000,104.100000,188.50000
mean area,569.0,654.889104,351.914129,143.500000,420.300000,551.100000,782.700000,2501.00000
mean smoothness,569.0,0.096360,0.014064,0.052630,0.086370,0.095870,0.105300,0.16340
mean compactness,569.0,0.104341,0.052813,0.019380,0.064920,0.092630,0.130400,0.34540
mean concavity,569.0,0.088799,0.079720,0.000000,0.029560,0.061540,0.130700,0.42680
mean concave points,569.0,0.048919,0.038803,0.000000,0.020310,0.033500,0.074000,0.20120
mean symmetry,569.0,0.181162,0.027414,0.106000,0.161900,0.179200,0.195700,0.30400
mean fractal dimension,569.0,0.062798,0.007060,0.049960,0.057700,0.061540,0.066120,0.09744


In [4]:
# Frequency of benign and malignant values
df['target'].value_counts()

target
1    357
0    212
Name: count, dtype: int64

In [5]:
# Splitting into train and test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=13)

# Use SVM classifier for binary classification
svc = SVC(C = 10, gamma = 0.01)
svc.fit(X_train, y_train)

# model predictions 
y_pred = svc.predict(X_test)

Now we can use Cyclops evaluation metrics to evaluate our model's performance. You can either use each metric individually by calling them, or define a ``MetricDict`` object.
Here, we show both methods.

### Individual Metrics
In case you need only a single metric, you can create an object of the desired metric and call it on your ground truth and predictions:

In [6]:
bin_acc_metric = BinaryAccuracy()
bin_acc_metric(y_test.values, np.float64(y_pred))

0.7192982456140351

### Using ``MetricDict``
You may define a collection of metrics in case you need more metrics. It also speeds up the metric calculation.

In [7]:
metric_names = [
    "binary_accuracy",
    "binary_precision",
    "binary_recall",
    "binary_f1_score",
    "binary_roc_curve",
    "binary_precision_recall_curve",
]
metrics = [
    create_metric(metric_name, experimental=True) for metric_name in metric_names
]
metric_collection = MetricDict(metrics)
metric_collection(y_test.values, np.float64(y_pred))

{'BinaryAccuracy': array(0.71929824, dtype=float32),
 'BinaryPrecision': array(0.7090909, dtype=float32),
 'BinaryRecall': array(1., dtype=float32),
 'BinaryF1Score': array(0.82978725, dtype=float32),
 'BinaryROC': (array([0.       , 0.8888889, 1.       ], dtype=float32),
  array([0., 1., 1.], dtype=float32),
  array([1., 1., 0.])),
 'BinaryPrecisionRecallCurve': (array([0.68421054, 0.7090909 , 1.        ], dtype=float32),
  array([1., 1., 0.], dtype=float32),
  array([0., 1.]))}

You may reset the metrics collection and add other metrics:

In [8]:
metric_collection.reset()
metric_collection.add_metrics(BinaryAveragePrecision(), BinaryAUROC())
metric_collection(y_test.values, np.float64(y_pred))

{'BinaryAccuracy': array(0.71929824, dtype=float32),
 'BinaryPrecision': array(0.7090909, dtype=float32),
 'BinaryRecall': array(1., dtype=float32),
 'BinaryF1Score': array(0.82978725, dtype=float32),
 'BinaryROC': (array([0.       , 0.8888889, 1.       ], dtype=float32),
  array([0., 1., 1.], dtype=float32),
  array([1., 1., 0.])),
 'BinaryPrecisionRecallCurve': (array([0.68421054, 0.7090909 , 1.        ], dtype=float32),
  array([1., 1., 0.], dtype=float32),
  array([0., 1.])),
 'BinaryAveragePrecision': 0.7090909,
 'BinaryAUROC': 0.5555556}

### Fairness Evaluation

In order to evaluate the fairness of your model, you have to set up a ``FairnessConfig`` object.

### Data Slicing

In addition to overall metrics, it might be interesting to see how the model performs on certain subpopulation or subsets. We can define these subsets using ``SliceSpec`` objects.

In [9]:
spec_list = [
    {
        "worst radius": {
            "min_value": 10.0,
            "max_value": 15.0,
            "min_inclusive": True,
            "max_inclusive": False,
        },
    },
    {
        "worst radius": {
            "min_value": 15.0,
            "max_value": 37.0,
            "min_inclusive": True,
            "max_inclusive": False,
        },
    },
]
slice_spec = SliceSpec(spec_list)

In [ ]:

# breast_cancer_data = Dataset.from_pandas(df)

# breast_cancer_sliced_result = evaluator.evaluate(
#     dataset=breast_cancer_data,
#     metrics=metric_collection,  # type: ignore[list-item]
#     target_columns='target',
#     prediction_columns=,
#     ignore_columns="image",
#     slice_spec=slice_spec,
# )
